In [ ]:
import os
import pandas as pd
import numpy as np
import subprocess


%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime
import json

from scipy import optimize
from scipy import integrate
from scipy import signal

from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

                 
import seaborn as sns
import requests
from bs4 import BeautifulSoup

import plotly.graph_objects as go
import plotly
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [ ]:
pop_df = pd.read_csv('../data/processed/population.csv',sep=';')
def getPopulationByCountry(country_name):
    return int(pop_df.iloc[0][country_name].replace(',',''))
pop = getPopulationByCountry('Afghanistan')

In [ ]:
confirmed_df = pd.read_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';')
recovered_df = pd.read_csv('../data/processed/COVID_small_flat_table_recovered.csv',sep=';')
deaths_df = pd.read_csv('../data/processed/COVID_small_flat_table_deaths.csv',sep=';')

In [ ]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';')  
df_analyse.sort_values('date',ascending=True).head()
df_analyse['index'] =np.arange(len(df_analyse['date']))
#df_analyse

In [ ]:
#Intialisation
ydata = np.array(df_analyse.Germany[35:])
t=np.arange(len(ydata))

N0 = 1000000
I0=ydata[0]
S0=N0-I0
R0=0
beta = 0.4

# SIR model

In [ ]:
def SIR_model(SIR,beta,gamma,N0):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])

# Propagation rates

In [ ]:
def getPropRates(country_name,country_population,infection_start,measures_start,hold,relax,beta_value,gamma_value):
    
    beta_max = beta_value 
    beta_min = 0.11
    gamma = gamma_value
    
    N0 = country_population
    I0 = df_analyse[country_name][infection_start]
    S0 = N0-I0
    R0 = 0
    
    t_initial = infection_start
    t_intro_measures = measures_start
    t_hold = hold
    t_relax = relax
    
    print(N0,I0,S0,t_initial,t_intro_measures)
    
    pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))
    
    SIR=np.array([S0,I0,R0])
    
    propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})
    
    for each_beta in pd_beta:
        new_delta_vec=SIR_model(SIR,each_beta,gamma,N0)
        SIR=SIR+new_delta_vec
        propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
    #print(propagation_rates['susceptible']) 
    return propagation_rates


# Fit curve

In [ ]:
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [ ]:
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
def fitParameters(ydata,timestep,population,beta,gamma):
    
    global N0
    N0 = population
    global I0
    I0 = ydata[0]
    global S0
    S0 = N0-I0
    global R0
    R0 = 0
    global t
    t = timestep
    
    popt=[beta,gamma]
    fit_odeint(t, *popt)
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))
    
    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
    
    # get the final fitted curve
    fitted=fit_odeint(t, *popt)
    return fitted,popt
ydata = np.array(df_analyse['Germany'][35:])
t=np.arange(len(ydata))
fitted,popt = fitParameters(ydata,t,getPopulationByCountry('Germany'),0.4,0.1)

In [ ]:
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fit of SIR model for Germany cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# Dashboard

In [ ]:
country_list = df_analyse.columns[1:]
country_dropdown=[]
for country in country_list:
    country_dropdown.append({'label':country,'value':country})
#country_list[0]

In [ ]:
ydata = df_analyse['Germany'][35:]
propagation_rates = getPropRates('Germany',1000000,60,70,100,100,0.2,0.1)

In [ ]:
df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')

In [ ]:
all_sim_fig = go.Figure()
all_sim_fig.add_trace(go.Scatter(x=propagation_rates.index,
                                y=propagation_rates['susceptible'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                marker_color = 'blue',
                                name='Susceptible'
                                 )
                     )

all_sim_fig.add_trace(go.Scatter(x=propagation_rates.index,
                                y=propagation_rates['recovered'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                marker_color = 'green',
                                name='Recovered'
                                 )
                     )
all_sim_fig.add_trace(go.Scatter(x=propagation_rates.index,
                                y=propagation_rates['infected'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                marker_color = 'red',
                                name='Infected'
                                 )
                     )
## defines the overall layout properties
all_sim_fig.update_layout(
    xaxis_title="No. of days",
    yaxis_title="Propagation rate",
    title = "SIR simulations"
)
all_sim_fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16
    )
)
all_sim_fig.update_xaxes(nticks = 15,showspikes=True)
all_sim_fig.update_yaxes(showspikes=True,type="log")
all_sim_fig.show()

In [ ]:
sim_fig = go.Figure()
sim_fig.add_trace(go.Bar(x=np.arange(len(ydata)),
                                y=ydata, 
                                name='Confirmed infected',
                                orientation = 'v',marker_color = '#FF5959'
                                 )
                     )
sim_fig.add_trace(go.Scatter(x=t,
                                y=fitted,
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                marker_color = '#39F09A',
                                name='Fitted curve - Optimal beta & gamma'
                                 )
                     )
sim_fig.add_trace(go.Scatter(x=propagation_rates.index,
                                y=propagation_rates['infected'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                marker_color = '#3A37EE',
                                name='Fitted curve - all parameters'
                                 )
                     )


## defines the overall layout properties
sim_fig.update_layout(
    xaxis_title="No. of days",
    yaxis_title="No. of cases",
    title = "Confirmed infected vs. dynamic parameters with SIR"
)
sim_fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16
    )
)
sim_fig.update_xaxes(nticks = 15,showspikes=True)
sim_fig.update_yaxes(showspikes=True,type="log")
sim_fig.show()

In [ ]:
confirmed_fig = go.Figure()
confirmed_fig.add_trace(go.Bar(x=confirmed_df['date'],
                                y=confirmed_df['Afghanistan'],
                                orientation = 'v',marker_color = 'grey'))

## defines the overall layout properties
confirmed_fig.update_layout(
    xaxis_title="Timeline",
    yaxis_title="No. of cases",
    title = "Total confirmed cases",
    xaxis_type='category'
)
confirmed_fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16
    )
)
confirmed_fig.update_xaxes(nticks = 5)
confirmed_fig.update_yaxes(showspikes=True)
confirmed_fig.show()

recovered_fig = go.Figure()
recovered_fig.add_trace(go.Scatter(x=recovered_df['date'],
                                y=recovered_df['Afghanistan'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Recovered'
                                 )
                     )

## defines the overall layout properties
recovered_fig.update_layout(
    xaxis_title="Timeline",
    yaxis_title="No. of cases",
    title = "Total recovered cases",
    xaxis_type='category'
)
recovered_fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16
    )
)
recovered_fig.update_xaxes(nticks = 5,showspikes=True,spikecolor="grey")
recovered_fig.update_yaxes(showspikes=True,spikecolor="grey")
recovered_fig.show()

deceased_fig = go.Figure()
deceased_fig.add_trace(go.Scatter(x=deaths_df['date'],
                                y=deaths_df['Afghanistan'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Deaths'
                                 )
                     )

## defines the overall layout properties
deceased_fig.update_layout(
    xaxis_title="Timeline",
    yaxis_title="No. of cases",
    title = "Total deaths",
    xaxis_type='category'
)
deceased_fig.update_layout(
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16
    )
)
deceased_fig.update_xaxes(nticks = 5,showspikes=True,spikecolor="grey")
deceased_fig.update_yaxes(showspikes=True,spikecolor="grey")
deceased_fig.show()

In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.H1('Dashboard for COVID-19 cases', style={"textAlign":'center'}),
    html.Br(),
    html.Div([
        html.P('Select Country'),
        dcc.Dropdown(id='country_dropdown', options=country_dropdown, value ='Afghanistan'),
        html.Br(),
        dbc.RadioItems(id='graph_type', options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                                       value='Linear',style={"display":'inline-block'}),
        
        dbc.Row([
                dbc.Col(dcc.Graph(figure=confirmed_fig, id='confirmed_fig_id')), 
                dbc.Col(dcc.Graph(figure=recovered_fig, id='recovered_fig_id')),
                dbc.Col(dcc.Graph(figure=deceased_fig, id='deceased_fig_id'))
        ])
    ]),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Br(),
        html.H2('SIR Simulation - enter the values for parameters',style={"textAlign":'center'}),
        html.Br(),
        dbc.FormGroup([
            dbc.Row([
                
                dbc.Col([
                    dbc.Label("Infection start (in days)"),
                    dbc.Input(id="infection_start", type="number", value=35)
                ]),
                dbc.Col([
                    dbc.Label("Intro measures start (in days)"),
                    dbc.Input(id="measures_start", type="number", value=40)
                ]),
                dbc.Col([
                    dbc.Label("Hold start (in days)"),
                    dbc.Input(id="hold_start", type="number", value=60),
                ]),
                dbc.Col([
                    dbc.Label("Relax start (in days)"),
                    dbc.Input(id="relax_start", type="number", value=60),
                    html.Br()
                    
                ]),
            ]),
            dbc.Row([
                
                dbc.Col([
                    dbc.Label("Beta value (infection rate)"),
                    dbc.Input(id="beta_value", type="number", value=0.4)
                ]),
                dbc.Col([
                    dbc.Label("Gamma value (death rate)"),
                    dbc.Input(id="gamma_value", type="number", value=0.1)
                ]),
                dbc.Col([
                    dbc.Label("Population"),
                    html.Div(id="population_div")
                ]),
                dbc.Col([
                    dbc.Label("Show simulation"),html.Br(),
                    dbc.Button("Submit", color="primary", id='submit_btn', n_clicks=0)
                ])
            ])
        ]),
        html.Div([
                dcc.Graph(figure=sim_fig, id='sim_fig_id'),
                dcc.Graph(figure=all_sim_fig, id='all_sim_fig_id')
        ])
    ]),
        
    html.Br(),
    html.Hr(),
    
    html.Div([
        html.Br(),
        html.H3('Confirmed COVID-19 cases and approximated doubling rates',style={"textAlign":'center'}),
        html.Br(),
        html.P('Select country'),
        dcc.Dropdown(id='country_multi_dropdown', options=country_dropdown, value=['US', 'Germany'],  multi=True),
        html.Br(),
        html.P('Select timeline'),
        dcc.Dropdown(id='doubling_time',
                     options=[
                         {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
                         {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
                         {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
                         {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'}
                     ],value='confirmed',multi=False),
        html.Br(),
        dcc.Graph(figure=recovered_fig, id='doubling_fig')
    ])
   
    
    
],style={"marginLeft":'2%',"marginRight":'2%',"marginTop":'2%'})

In [ ]:
@app.callback(
    Output('confirmed_fig_id', 'figure'),
    [Input('country_dropdown', 'value'),
     Input('graph_type', 'value')])
def update_figure(country_name,graph_type):
    return {
            'data': [dict(x=confirmed_df['date'],
                                y=confirmed_df[country_name],
                                orientation = 'v',marker = {"color": "#f7786b"},type='bar')],
            'layout': dict (
                xaxis={'tickangle':-45,
                        'nticks':5,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        'title' : 'Timeline'
                        
                      },
                yaxis={
                       'range':'[1.1,5.5]',
                       'title':"No. of cases",
                       'type':'linear' if graph_type == 'Linear' else 'log'
                      },
                title = "Total confirmed cases for "+country_name
        )
    }
@app.callback(
    Output('recovered_fig_id', 'figure'),
    [Input('country_dropdown', 'value'),
     Input('graph_type', 'value')])
def update_figure(country_name,graph_type):
    return {
            'data': [dict(x=recovered_df['date'],
                                y=recovered_df[country_name],
                                orientation = 'v',marker = {"color": "#82b74b"},type='bar')],
            'layout': dict (
                xaxis={'nticks':5,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        'title' : 'Timeline'
                        
                      },
                yaxis={
                       'range':'[1.1,5.5]',
                       'title':"No. of cases",
                       'type':'linear' if graph_type == 'Linear' else 'log'
                      },
                title = "Total recovered cases for "+country_name
        )
    }
@app.callback(
    Output('deceased_fig_id', 'figure'),
    [Input('country_dropdown', 'value'),
     Input('graph_type', 'value')])
def update_figure(country_name,graph_type):
    return {
            'data': [dict(x=deaths_df['date'],
                                y=deaths_df[country_name],
                                orientation = 'v',marker = {"color": "grey"},type='bar')],
            'layout': dict (
                xaxis={'nticks':5,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        'title' : 'Timeline'
                        
                      },
                yaxis={
                       'range':'[1.1,5.5]',
                       'title':"No. of cases",
                       'type':'linear' if graph_type == 'Linear' else 'log'
                      },
                title = "Total deaths for "+country_name
        )
    }

In [ ]:
@app.callback(
    Output('population_div', 'children'),
    [Input('country_dropdown', 'value')])
def update_population(country_name):
    pop = getPopulationByCountry(country_name)
    return dbc.Label(pop)

In [ ]:
@app.callback(
    [   dash.dependencies.Output('sim_fig_id', 'figure'),
        dash.dependencies.Output('all_sim_fig_id', 'figure')],
    [
        dash.dependencies.Input('country_dropdown', 'value'),
        dash.dependencies.Input('submit_btn','n_clicks')],
    [
        dash.dependencies.State('infection_start', 'value'),
        dash.dependencies.State('measures_start', 'value'),
        dash.dependencies.State('hold_start', 'value'),
        dash.dependencies.State('relax_start', 'value'),
        dash.dependencies.State('beta_value', 'value'),
        dash.dependencies.State('gamma_value', 'value')
    ])
def update_figure(country_name,n_clicks,infection_start,measures_start,hold,relax,beta_value,gamma_value):
    country_population = getPopulationByCountry(country_name)
    ydata = np.array(df_analyse[country_name][infection_start:])
    t=np.arange(len(ydata))
    fitted,popt = fitParameters(ydata,t,country_population,beta_value,gamma_value)
    propagation_rates = getPropRates(country_name,country_population,infection_start,measures_start,hold,relax,beta_value,gamma_value)
    optimal = "Optima beta("+str(popt[0])+" & gamma ("+str(popt[1])
    return {
            'data': [dict(x=t,
                                y=ydata,
                                type='bar',
                                name='Actual infected',
                                marker = {"color": "#b1cbbb"}
                         ),
                    dict(x=propagation_rates.index,
                                y=propagation_rates['infected'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Fitted curve - all parameters', marker = {"color": "#d64161"}),
                     dict(x=t,
                                y=fitted,
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Fitted curve - Optimal beta & gamma',marker = {"color": "#6b5b95"})],
            'layout': dict (
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        'title' : 'No. of days'
                        
                      },
                yaxis={
                       'range':'[1.1,5.5]',
                       'title':"No. of cases",
                       'type':'log'
                      },
                title = "Acutal infected vs. Dynamic parameters for SIR (Optimal beta - "+str("%.2f" % popt[0])+" & gamma - "+str("%.2f" % popt[1])+")"
        )
    },{
        'data': [dict(x=propagation_rates.index,
                                y=propagation_rates['susceptible'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Susceptible', marker = {"color": "#ff7b25"}),
                    dict(x=propagation_rates.index,
                                y=propagation_rates['infected'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Infected', marker = {"color": "#ff6f69"}),
                     dict(x=propagation_rates.index,
                                y=propagation_rates['recovered'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name='Recovered',marker = {"color": "#82b74b"}
                         )],
            'layout': dict (
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        'title' : 'No. of days'
                        
                      },
                yaxis={
                       'range':'[1.1,5.5]',
                       'title':"Propagation rates",
                       'type':'log'
                      },
                title = "Dynamic parameters with SIR"
        )
        
    
   }

In [ ]:
@app.callback(
    Output('doubling_fig', 'figure'),
    [Input('country_multi_dropdown', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people(log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }


In [ ]:
app.run_server(debug=True, use_reloader=False)